Broadcasting
====

# ufunc-broadcasting

Each universal function takes array inputs and produces array outputs by performing the core function element-wise on the inputs (where an element is generally a scalar, but can be a vector or higher-order sub-array for generalized ufuncs). Standard broadcasting rules are applied so that inputs not sharing exactly the same shapes can still be usefully operated on. Broadcasting can be understood by four rules:

1. All input arrays with ndim smaller than the input array of largest ndim, have 1’s prepended to their shapes.
2. The size in each dimension of the output shape is the maximum of all the input sizes in that dimension.
3. An input can be used in the calculation if its size in a particular dimension either matches the output size in that dimension, or has value exactly 1.
4. If an input has a dimension size of 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension. In other words, the stepping machinery of the ufunc will simply not step along that dimension (the stride will be 0 for that dimension).

每个通用函数接受数组输入并通过在输入上逐元素地执行核心函数来生成数组输出（其中的元素通常是标量，但可以是用于广义ufunc的向量或更高阶子数组）。应用标准广播规则，以便仍然可以有效地操作不共享完全相同形状的输入。广播可以通过四个规则来理解：

1. 所有输入数组，比最大的 ndim 小的 ndim 输入数组，对它们的形状有一个预处理。
2. 输出形状的每个维度的 size 是该维度中所有输入 size 的最大值。
3. 一个输入的特定维度的 size 可以用于计算，要么匹配该维度的输出 size，要么其值正好是 1。
4. 如果一个输入的 shape 中的一个维度的 size 为 1，则该维度中的第一个数据条目将用于沿该维度的所有计算。换句话说，ufunc的步进器 将不会沿着该维度步进（对于该维度， 步幅将为0）。

Broadcasting is used throughout NumPy to decide how to handle disparately shaped arrays; for example, all arithmetic operations `(+, -, *, …)` between ndarrays broadcast the arrays before operation.

A set of arrays is called “broadcastable” to the same shape if the above rules produce a valid result, i.e., one of the following is true:

整个 NumPy 使用广播来决定如何处理不同形状的数组; 例如，所有算术运算`（+， -，*，...）`ndarrays的阵列操作之前广播。

如果上述规则产生有效结果，则将一组数组称为“可广播 ”到相同的形状，即，满足以下条件之一：

1. The arrays all have exactly the same shape.
2. The arrays all have the same number of dimensions and the length of each dimensions is either a common length or 1.
3. The arrays that have too few dimensions can have their shapes prepended with a dimension of length 1 to satisfy property 2.

1. 阵列都具有完全相同的形状。
2. 阵列都具有相同的维数，每个维度的长度是“通约的”长度或1。
3. 具有太少维度的数组可以用长度为 1 的维度对其shapes预处理以满足属性2。

Example
If `a.shape` is `(5,1)`, `b.shape` is `(1,6)`, `c.shape` is `(6,)` and `d.shape` is `()` so that d is a scalar, then a, b, c, and d are all broadcastable to dimension (5,6); and

例如：

如果 `a.shape` 是 `(5,1)`, `b.shape` 是 `(1,6)`, `c.shape` 是 `(6,)` and `d.shape` is `()`，这样 d 是一个标量，而 a, b, c, and d 都是可以广播到维度(5,6); 并且：

- a acts like a `(5,6)` array where `a[:,0]` is broadcast to the other columns,
- b acts like a `(5,6)` array where `b[0,:]` is broadcast to the other rows,
- c acts like a `(1,6)` array and therefore like a `(5,6)` array where `c[:]` is broadcast to every row, and finally,
- d acts like a `(5,6)` array where the single value is repeated.

- a 的行为类似数组 `(5,6)`，  `a[:,0]`被广播到其它列,
- b 的行为类似数组 `(5,6)`， `b[0,:]` 被广播到其它行，
- c 的行为类似数组 `(1,6)`，因此还类似于数组 `(5,6)`，`c[:]`被广播到所有行，并结束,
- d 的行为类似数组 `(5,6)`，单个值被重复。

# numpy.doc.broadcasting

The term broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

NumPy operations are usually done on pairs of arrays on an element-by-element basis. In the simplest case, the two arrays must have exactly the same shape, as in the following example:

术语广播描述了numpy在算术运算时如何处理具有不同形状的数组。受限于某些约束性条件，较小的数组在较大的数组上“广播”，因此它们应有“兼容”的形状。广播提供了一种矢量化数组操作的方法，以便在C而不是Python中进行循环。它可以在不产生不必要的数据副本的情况下实现这一点，并且通常可以更高效，然而，有些情况下广播是一个坏主意，因为它会导致内存使用效率低下，从而减慢计算速度。

在成对的数组上，NumPy操作通常基于逐个元素执行。在最简单的情况下，两个数组必须具有完全相同的形状，如下例所示：

In [17]:
import numpy as np
>>> a = np.array([1.0, 2.0, 3.0])
>>> b = np.array([2.0, 2.0, 2.0])
>>> a * b

array([2., 4., 6.])

NumPy’s broadcasting rule relaxes this constraint when the arrays’ shapes meet certain constraints. The simplest broadcasting example occurs when an array and a scalar value are combined in an operation:

当阵列的形状满足某些特殊的约束时，NumPy的广播规则放宽了这种约束。当一个数组和一个标量值被组合在一个操作中时，会发生最简单的广播示例：

In [18]:
>>> a = np.array([1.0, 2.0, 3.0])
>>> b = 2.0
>>> a * b

array([2., 4., 6.])

The result is equivalent to the previous example where b was an array. We can think of the scalar b being stretched during the arithmetic operation into an array with the same shape as a. The new elements in b are simply copies of the original scalar. The stretching analogy is only conceptual. NumPy is smart enough to use the original scalar value without actually making copies, so that broadcasting operations are as memory and computationally efficient as possible.

The code in the second example is more efficient than that in the first because broadcasting moves less memory around during the multiplication (b is a scalar rather than an array).

结果等同于前面的例子，在前面的例子中 `b` 是一个数组。我们可以将在算术运算期间被拉伸的标量`b`想象成具有与`a`相同形状的数组。`b`中的新元素只是原始标量的副本。 拉伸类比只是概念性的。 NumPy足够智能，可以使用原始标量值而无需实际制作副本，因此广播操作尽可能提高内存和计算效率。

第二个例子中的代码比第一个例子中的代码更有效，因为广播在乘法过程中移动的内存较少（b是标量而不是数组）。

## General Broadcasting Rules

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when

1. they are equal, or
2. one of them is 1

在两个阵列上运行时，NumPy会逐元素地比较它们的形状。它从尾部维度开始，并向前工作。 在满足下面的两个条件时，两个维度是兼容的：

1. 他们是相等的，或者
2. 其中一个是1

If these conditions are not met, a ValueError: frames are not aligned exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the maximum size along each dimension of the input arrays.

Arrays do not need to have the same number of dimensions. For example, if you have a 256x256x3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3
```

如果不满足这些条件，则会抛出`ValueError：frames are not aligned`（值错误：框架未对齐）的异常，表示数组具有不兼容的形状。**结果数组的大小是沿输入数组的每个维度的最大大小。**

数组不需要具有相同数量的维度。例如，如果您有一个 `256x256x3` 的RGB值数组，并且您希望将图像中的每种颜色缩放不同的值，则可以将图像乘以具有3个值的一维数组。 根据广播规则排列这些数组的尾轴的大小，下面表明它们是兼容的：

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3
```

When either of the dimensions compared is one, the other is used. In other words, dimensions with size 1 are stretched or “copied” to match the other.

In the following example, both the A and B arrays have axes with length one that are expanded to a larger size during the broadcast operation:

当比较的任何一个维度为`1`时，使用另一个维度。 换句话说，维度为`1`的维度被拉伸或“复制”以匹配另一个维度。

在以下示例中，`A` 和 `B` 阵列都具有长度为`1`的轴，在广播操作期间将其扩展为更大的大小：

```
A     (4d array):    8 x 1 x 6 x 1
B     (3d array):        7 x 1 x 5
Result  (4d array):  8 x 7 x 6 x 5
```

Here are some more examples:

下面是一些示例：

```
A      (2d array):  5 x 4
B      (1d array):      1
Result (2d array):  5 x 4

A      (2d array):  5 x 4
B      (1d array):      4
Result (2d array):  5 x 4

A      (3d array):  15 x 3 x 5
B      (3d array):  15 x 1 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 1
Result (3d array):  15 x 3 x 5
```

Here are examples of shapes that do not broadcast:

下面是形状不广播的示例：

```
A      (1d array):  3
B      (1d array):  4 # trailing dimensions do not match

A      (2d array):      2 x 1
B      (3d array):  8 x 4 x 3 # second from last dimensions mismatched
```

An example of broadcasting in practice:

实践中广播的例子：

In [19]:
>>> x = np.arange(4)
>>> xx = x.reshape(4,1)
>>> y = np.ones(5)
>>> z = np.ones((3,4))

In [20]:
>>> x.shape

(4,)

In [21]:
y.shape

(5,)

In [22]:
x + y

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [ ]:
xx.shape

In [ ]:
y

In [ ]:
xx

In [ ]:
(xx + y).shape

In [ ]:
xx + y

In [ ]:
x.shape

In [ ]:
z.shape

In [ ]:
x + z

Broadcasting provides a convenient way of taking the outer product (or any other outer operation) of two arrays. The following example shows an outer addition operation of two 1-d arrays:

广播提供了一种方便的方式来获取两个数组的外部产品（或任何其他外部操作）。 以下示例显示了两个1-d数组的外部加法操作：

In [ ]:
>>> a = np.array([0.0, 10.0, 20.0, 30.0])
>>> b = np.array([1.0, 2.0, 3.0])
>>> a[:, np.newaxis] + b

In [29]:
a

array([1., 2., 3.])

In [30]:
b

Here the newaxis index operator inserts a new axis into a, making it a two-dimensional `4x1` array. Combining the `4x1` array with b, which has shape `(3,)`, yields a 4x3 array.

这里newaxis索引操作符将一个新轴插入a，使其成为一个二维4x1数组。 将`4x1`数组与具有形状（3，）的`b`组合，产生`4x3`数组。

# numpy.broadcast

[`class numpy.broadcast`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.broadcast.html#numpy.broadcast)

Produce an object that mimics broadcasting.

产生一个模仿广播的对象。

## Parameters:	
**in1, in2, … **: array_like

Input parameters. 输入参数。

## Returns:	
b : broadcast object,  广播对象。

将输入参数相互广播，并返回封装结果的对象。 其中，它具有shape和nd属性，可以用作迭代器。

Broadcast the input parameters against one another, and return an object that encapsulates the result. Amongst others, it has shape and nd properties, and may be used as an iterator.

**See also**

`broadcast_arrays, broadcast_to`

## Examples

使用广播手动添加两个向量：

Manually adding two vectors, using broadcasting:

In [ ]:
>>> x = np.array([[1], [2], [3]])
>>> y = np.array([4, 5, 6])
>>> b = np.broadcast(x, y)

In [ ]:
>>> out = np.empty(b.shape)
>>> out.flat = [u+v for (u,v) in b]
>>> out

与内置广播比较：

Compare against built-in broadcasting:

In [ ]:
>>> x + y

## Attributes:

**index**

current index in broadcasted result

**iters**

tuple of iterators along self’s “components.”

**nd**

广播结果的维数。 预期对于 NumPy 1.12.0 及其更高版本的代码，首选更一致的 ndim 。

Number of dimensions of broadcasted result. For code intended for NumPy 1.12.0 and later the more consistent ndim is preferred.

In [23]:
>>> x = np.array([1, 2, 3])
>>> y = np.array([[4], [5], [6]])
>>> b = np.broadcast(x, y)
>>> b.nd

2

**ndim**

广播结果的维数。 别名为nd。

Number of dimensions of broadcasted result. Alias for nd.

New in version 1.12.0.

In [24]:
>>> x = np.array([1, 2, 3])
>>> y = np.array([[4], [5], [6]])
>>> b = np.broadcast(x, y)
>>> b.ndim

2

**numiter**

广播结果拥有的迭代器数量。

Number of iterators possessed by the broadcasted result.

In [25]:
>>> x = np.array([1, 2, 3])
>>> y = np.array([[4], [5], [6]])
>>> b = np.broadcast(x, y)
>>> b.numiter

2

**shape**

Shape of broadcasted result.

**size**

Total size of broadcasted result.

## Methods

`reset()`	Reset the broadcasted result’s `iterator(s)`.

# numpy.broadcast_arrays
`numpy.broadcast_arrays(*args, **kwargs)`

Broadcast any number of arrays against each other.

## Parameters:	
**`*args`** : array_likes

The arrays to broadcast.

**subok** : bool, optional

如果为True，则子类将被传递，否则返回的数组将被强制为基类数组（默认）。

If True, then sub-classes will be passed-through, otherwise the returned arrays will be forced to be a base-class array (default).

## Returns:	
**broadcasted** : list of arrays

这些数组是原始数组的视图。 它们通常不是连续的。 此外，广播阵列的多于一个元素可以指代单个存储位置。 如果需要写入数组，请先进行复制。

These arrays are views on the original arrays. They are typically not contiguous. Furthermore, more than one element of a broadcasted array may refer to a single memory location. If you need to write to the arrays, make copies first.

## 示例

In [26]:
>>> x = np.array([[1,2,3]])
>>> y = np.array([[1],[2],[3]])
>>> np.broadcast_arrays(x, y)

[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]), array([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])]

Here is a useful idiom for getting contiguous copies instead of non-contiguous views.
这是获取连续副本而非非连续视图的有用习惯用法。

In [27]:
>>> [np.array(a) for a in np.broadcast_arrays(x, y)]

[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]), array([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])]

# numpy.broadcast_to
`numpy.broadcast_to(array, shape, subok=False)`

Broadcast an array to a new shape.

## Parameters:	
**array** : array_like

The array to broadcast.

**shape** : tuple
The shape of the desired array.

**subok** : bool, optional

如果为True，则子类将被传递，否则返回的数组将被强制为基类数组（默认）。

If True, then sub-classes will be passed-through, otherwise the returned array will be forced to be a base-class array (default).

## Returns:	
**broadcast** : array

具有给定形状的原始数组的只读视图。 它通常不是连续的。 此外，广播阵列的多于一个元素可以指代单个存储位置。

A readonly view on the original array with the given shape. It is typically not contiguous. Furthermore, more than one element of a broadcasted array may refer to a single memory location.

## Raises:	
**ValueError**

根据NumPy的广播规则，如果阵列与新形状不兼容。

If the array is not compatible with the new shape according to NumPy’s broadcasting rules.

## 示例

In [28]:
>>> x = np.array([1, 2, 3])
>>> np.broadcast_to(x, (3, 3))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])